In [1]:
import torch
from torch.autograd import Variable
from torchvision.transforms import transforms
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [3]:
weight = "C:/Users/Abhis/Desktop/res10_300x300_ssd_iter_140000.caffemodel"
prototxt = "C:/Users/Abhis/Desktop/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(prototxt, weight)

Basemodel=torch.load(r'C:\Users\Abhis\Desktop\detector\MaskDetStuff\model.pth', map_location=torch.device('cpu'))

#Basemodel=torch.load('C:/Users/Abhis/Desktop/modelNoise.pth',map_location=torch.device('cpu'))
Basemodel.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [70]:
img=Image.open("C:/Users/Abhis/Pictures/Screenshots/Screenshot (721).png").convert('RGB')
pred(img, tsfm)

'With_Mask'

In [4]:
classes = ['With_Mask', 'Without_Mask']

tsfm = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # mean across each RGB channel
                         [0.229, 0.224, 0.225])
])

def pred(img, tsfm):
    tensor_img = tsfm(img)

    tensor_img = torch.unsqueeze(tensor_img,0) #only for single image pred rather than batch pred
    

    imginput = Variable(tensor_img)

    prediction = Basemodel(imginput)
    index = prediction.data.numpy().argmax()
    #print(index)
    output = classes[index]

    return output

In [5]:
import time
import sys

In [7]:

cap=cv2.VideoCapture(0)
while cap.isOpened():
    
    
    _, frame=cap.read()
    

    #DNN:
    img = frame.copy()
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),1.0, 
                                                   (300, 300), 
                                                   (104.0, 117.0, 123.0))
    height, width=frame.shape[:2]
    net.setInput(blob)
    faces = net.forward()
    ender=0
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.4:
            box = faces[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x, y, x1, y1) = box.astype("int")
        
            #print(x,y,x1,y1)
            roi_color=img[y:y1, x:x1]
        
            roi_resize = cv2.resize(roi_color, (224, 224))
        #print(roi_resize)
            starter=time.time()
            output=pred(roi_resize, tsfm)
            ender=time.time()
        #print(output)
        #output='With_Mask'
        
            label = "Masked" if output=='With_Mask' else "Not Masked!"
        
            colour = (0, 255, 0) if label == "Masked" else (0, 0, 255)
        
            cv2.putText(frame, label, (x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.4, colour, 2)
        
            cv2.rectangle(frame, (x, y), (x1, y1), colour, 4)
    
    cv2.imshow("Webcam Feed",frame)

    if cv2.waitKey(1) & 0xFF == ord('f'):
        break
    
    print("Inference Time", ender - starter)
        
cap.release()
cv2.destroyAllWindows()
    

Inference Time -1667318092.7642307
Inference Time -1667318092.7642307
Inference Time -1667318092.7642307
Inference Time 0.0947115421295166
Inference Time 0.0797872543334961
Inference Time 0.09275269508361816
Inference Time 0.0718071460723877
Inference Time 0.08876276016235352
Inference Time 0.08477568626403809
Inference Time 0.0937495231628418
Inference Time 0.09175491333007812
Inference Time 0.09275031089782715
Inference Time 0.0827791690826416
Inference Time 0.09574294090270996
Inference Time 0.09175419807434082
Inference Time 0.0857706069946289
Inference Time 0.09175515174865723
Inference Time 0.08876347541809082
Inference Time 0.08676767349243164
Inference Time 0.08776521682739258
Inference Time 0.0967411994934082
Inference Time 0.09275388717651367
Inference Time 0.09175515174865723
Inference Time 0.0967411994934082
Inference Time 0.0827796459197998
Inference Time 0.10073137283325195
Inference Time 0.0907583236694336
Inference Time 0.08676886558532715
Inference Time 0.0757970809936